In [8]:
# ==============================
# 1. Import Library
# ==============================
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Optional: miceforest (hanya jika ingin imputasi model)
try:
    import miceforest as mf
    MICE_AVAILABLE = True
except ImportError:
    MICE_AVAILABLE = False
    print("Library miceforest belum diinstal. Lewati tahap imputasi model ini.")


# ==============================
# 2. Load Dataset
# ==============================
features = pd.read_csv("features.csv")
labels = pd.read_csv("labels.csv")

print("Data features.csv dan labels.csv berhasil dimuat.\n")

print("=== Features (5 baris pertama) ===")
display(features.head())
print("\n=== Labels (5 baris pertama) ===")
display(labels.head())


# ==============================
# 3. Penjelasan Kolom
# ==============================
print("\nDeskripsi Kolom Features:")
print(features.info())

print("\nDeskripsi Kolom Labels:")
print(labels.info())


# ==============================
# 4. Agregasi antara features dan labels
# ==============================
# Gabungkan berdasarkan kolom yang sama (jika ada)
common_cols = set(features.columns).intersection(labels.columns)
if common_cols:
    key = list(common_cols)[0]
    data = pd.merge(features, labels, on=key, how="inner")
    print(f"Data berhasil digabung berdasarkan kolom '{key}'.")
else:
    data = pd.concat([features.reset_index(drop=True),
                      labels.reset_index(drop=True)], axis=1)
    print("Tidak ada kolom kunci yang sama. Data digabung berdasarkan index.")

print("\nData Gabungan (5 baris pertama):")
display(data.head())


# ==============================
# 5. Tipe Data Setiap Kolom
# ==============================
print("\nTipe data setiap kolom:")
print(data.dtypes)


# ==============================
# 6. Statistik Deskriptif
# ==============================
print("\nStatistik deskriptif untuk kolom numerik:")
display(data.describe(include=[np.number]).T)

print("\nStatistik deskriptif untuk kolom kategorikal:")
display(data.describe(include=[object]).T)

print("\nAnalisis singkat:")
print("- Statistik deskriptif menunjukkan nilai mean, std, dan quartile untuk memahami distribusi data.")
print("- Perhatikan nilai minimum, maksimum, serta kemungkinan adanya data kosong atau outlier.")


# ==============================
# 7. Slicing Data
# ==============================
print("\n20 baris pertama:")
display(data.head(20))

print("\n20 baris terakhir:")
display(data.tail(20))

if len(data) > 100:
    print("\nBaris index 80 - 100:")
    display(data.iloc[80:101])
else:
    print("\nDataset memiliki kurang dari 100 baris, tidak dapat menampilkan index 80–100.")


# ==============================
# 8. Tangani Nilai Hilang (NaN)
# ==============================
print("\nJumlah nilai hilang sebelum imputasi:")
print(data.isnull().sum())

# Pisahkan kolom numerik dan kategorikal
num_cols = data.select_dtypes(include=[np.number]).columns
obj_cols = data.select_dtypes(include=['object']).columns

# Imputasi numerik (gunakan median / mean / mode)
imputer_num = SimpleImputer(strategy='median')
data[num_cols] = imputer_num.fit_transform(data[num_cols])

# Imputasi kategorikal (gunakan modus)
for col in obj_cols:
    mode_value = data[col].mode()[0]
    data[col] = data[col].fillna(mode_value)

print("\nJumlah nilai hilang setelah imputasi:")
print(data.isnull().sum())


# ==============================
# 9. Encoding (opsional)
# ==============================
use_encoding = "label"  # ubah ke 'onehot' jika ingin OneHotEncoder

if use_encoding == "label":
    print("\nMenggunakan LabelEncoder untuk kolom kategorikal.")
    le = LabelEncoder()
    for col in obj_cols:
        data[col] = le.fit_transform(data[col].astype(str))
elif use_encoding == "onehot":
    print("\nMenggunakan OneHotEncoder untuk kolom kategorikal.")
    data = pd.get_dummies(data, columns=obj_cols)

print("\nContoh hasil setelah encoding:")
display(data.head())


# ==============================
# 10. Hasil Akhir Dataset
# ==============================
print("\nDataset akhir setelah preprocessing:")
display(data.head())

# Simpan hasil ke file CSV
data.to_csv("preprocessed_output.csv", index=False)
print("\nFile hasil preprocessing disimpan sebagai preprocessed_output.csv")


Library miceforest belum diinstal. Lewati tahap imputasi model ini.
Data features.csv dan labels.csv berhasil dimuat.

=== Features (5 baris pertama) ===


,Unnamed: 0,Student Id,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,0,6634,single,1.0,3.0,9773.0,day,1.0,NaN,Portuguese,...,0.0,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02
1,1,6459,single,1.0,1.0,NaN,day,1.0,136.0,Portuguese,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79
2,2,2238,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,...,0.0,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70
3,3,2479,single,1.0,1.0,NaN,day,1.0,141.0,Portuguese,...,0.0,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02
4,4,4260,single,44.0,1.0,NaN,day,39.0,120.0,Spanish,...,0.0,1.0,6.0,NaN,3.0,12.000000,0.0,12.4,0.5,1.79



=== Labels (5 baris pertama) ===


,Target
0,Graduate
1,Graduate
2,Graduate
3,Graduate
4,Dropout



Deskripsi Kolom Features:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Unnamed: 0                                      3096 non-null   int64  
 1   Student Id                                      3096 non-null   int64  
 2   Marital status                                  3012 non-null   object 
 3   Application mode                                2924 non-null   float64
 4   Application order                               2988 non-null   float64
 5   Course                                          2944 non-null   float64
 6   Daytime/evening attendance	                     3016 non-null   object 
 7   Previous qualification                          2973 non-null   float64
 8   Previous qualification (grade)                  3011 non-null   float64
 9   Nacionality   

...


Jumlah nilai hilang sebelum imputasi:
Unnamed: 0                                          0
Student Id                                          0
Marital status                                     84
Application mode                                  172
Application order                                 108
Course                                            152
Daytime/evening attendance\t                       80
Previous qualification                            123
Previous qualification (grade)                     85
Nacionality                                       185
Mother's qualification                            145
Father's qualification                             96
Mother's occupation                                62
Father's occupation                               144
Admission grade                                   141
Displaced                                         122
Educational special needs                         119
Debtor                                     

,Unnamed: 0,Student Id,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,6634.0,4,1.0,3.0,9773.0,0,1.0,133.1,13,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,2
1,1.0,6459.0,4,1.0,1.0,9238.0,0,1.0,136.0,13,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,2
2,2.0,2238.0,4,42.0,1.0,9500.0,0,1.0,120.0,13,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,2
3,3.0,2479.0,4,1.0,1.0,9238.0,0,1.0,141.0,13,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,2
4,4.0,4260.0,4,44.0,1.0,9238.0,0,39.0,120.0,17,...,1.0,6.0,8.0,3.0,12.000000,0.0,12.4,0.5,1.79,0



Dataset akhir setelah preprocessing:


,Unnamed: 0,Student Id,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,6634.0,4,1.0,3.0,9773.0,0,1.0,133.1,13,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,2
1,1.0,6459.0,4,1.0,1.0,9238.0,0,1.0,136.0,13,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,2
2,2.0,2238.0,4,42.0,1.0,9500.0,0,1.0,120.0,13,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,2
3,3.0,2479.0,4,1.0,1.0,9238.0,0,1.0,141.0,13,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,2
4,4.0,4260.0,4,44.0,1.0,9238.0,0,39.0,120.0,17,...,1.0,6.0,8.0,3.0,12.000000,0.0,12.4,0.5,1.79,0



File hasil preprocessing disimpan sebagai preprocessed_output.csv
